In [15]:
import heapq
import random

random.seed(20)

# Declare the Puzzle Node Class
The Puzzle Node has following attributes: <br>
 - Board : contains the 2D matrix for board <br>
 - Parent : contains the parend node of the board <br>
 - g : contains the cost to reach the node <br>
 - h : contains the heuristic value of the node <br>
 - f : contains the total cost of the node , f = g + h <br>

In [16]:
class PuzzleNode:
    def __init__(self, board, parent=None, move=None):
        """
        board: list of lists
        parent: PuzzleNode
        move: tuple
        """
        self.board = board
        self.parent = parent
        self.g = 0  # cost from start to current node
        self.h = 0  # heuristic value
        self.f = 0  # total cost

    def __lt__(self, other):
        return self.f < other.f

# Check Solvability
Not all initial boards can lead to the goal board by a sequence of  legal moves. We can whetther a configuration is solvable without attempting to solve it if we use inversion. Given a board, an inversion is any pair of blocks $𝑖$ and $𝑗$
where $𝑖 < 𝑗$ but $𝑖$ appears after $𝑗$ when considering the board in row-major order (row 0, followed 
by row 1, and so forth). 
- If the grid size $𝑘$ is an odd integer, then each legal move changes the number of inversions by an 
even number. Thus, if a board has an odd number of inversions, then it cannot lead to the goal 
board by a sequence of legal moves because the goal board has an even number of inversions 
(zero). The converse is also true, if a board has an even number of inversions, then it can lead to 
the goal board by a sequence of legal moves. 
- If the grid size 𝑘 is an even integer, puzzle instance is solvable if <br>
&emsp; - the blank is on an even row counting from the bottom (second-last, fourth-last, etc.) and 
number of inversions is odd. <br> 
&emsp; - the blank is on an odd row counting from the bottom (last, third-last, fifth-last, etc.) and 
number of inversions is even. <br>
&emsp; - For all other cases, the puzzle instance is not solvable.

In [17]:
def count_inversion(init_matrix):
    # Inversion count is the occurrence of smaller numbers after a certain number
    inversion_count = 0
    array_of_num = []

    # Converting to a 1D array
    for i in range(len(init_matrix)):
        for j in range(len(init_matrix[i])):
            array_of_num.append(init_matrix[i][j])

    # Iterating over the array
    for i in range(len(array_of_num)):
        x = array_of_num[i]
        for j in range(i + 1, len(array_of_num)):
            if array_of_num[j] < x and array_of_num[j] != 0:  # If there is any smaller number after index of x
                inversion_count += 1

    return inversion_count

def is_solvable(size, init_matrix):
    if size % 2 != 0:  # If grid size is odd
        if count_inversion(init_matrix) % 2 == 0:  # If inversion is even -> solvable
            return True
        else:  # If inversion is odd -> not solvable
            return False
    else:  # If grid size is even
        blank_pos = [(i, j) for i, row in enumerate(init_matrix) for j, num in enumerate(row) if num == 0][0]

        if (blank_pos[0] % 2 == 0) and (count_inversion(init_matrix) % 2 != 0):  # If blank is in even row and inversion count is odd -> solvable
            return True
        elif (blank_pos[0] % 2 != 0) and (count_inversion(init_matrix) % 2 == 0):  # If blank is in odd row and inversion count is even -> solvable
            return True
        else:
            return False

# Necessary Functions

In [18]:
def find_blank(board):
    """
    finds the blank tile in the board
    logic : if the tile is 0, it is the blank tile
    params :
        board : list of lists
    """
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] == 0:
                return i, j

def is_valid_move(i, j, n):
    """
    checks if the move is valid
    logic : if the move is within the board, it is valid
    params :
        i : int
        j : int
        n : int
    """
    return 0 <= i < n and 0 <= j < n

def get_valid_moves(i, j, n):
    """
    returns the valid moves for the blank tile
    logic : if the move is within the board, it is valid
    params :
        i : int
        j : int
        n : int
    """
    moves = []
    for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        ni, nj = i + dx, j + dy
        if is_valid_move(ni, nj, n):
            moves.append((ni, nj))
    return moves

def generate_neighbors(node):
    """
    generates the neighbors of the node
    logic : for each valid move, swap the blank tile with the tile in the move
    params :
        node : PuzzleNode
    """
    neighbors = []
    blank_i, blank_j = find_blank(node.board)
    valid_moves = get_valid_moves(blank_i, blank_j, len(node.board))

    for i, j in valid_moves:
        new_board = [row[:] for row in node.board]
        new_board[blank_i][blank_j], new_board[i][j] = new_board[i][j], new_board[blank_i][blank_j]
        neighbors.append(PuzzleNode(new_board, node, (i, j)))
    
    return neighbors

def reconstruct_path(node):
    """
    reconstructs the path from the node to the root
    logic : traverse the tree from the node to the root
    params :
        node : PuzzleNode
    """
    path = []
    current = node
    while current is not None:
        path.append(current.board)
        current = current.parent
    path.reverse()
    return path

# Heuristics
### Hamming Distance
Hamming distance is the number of tiles that are out of place. For example, the Hamming distance between
```
1 2 3
4 5 6
7 8 0
```
and
```
1 2 3
4 5 6
8 7 0
```
is 2 because 7 and 8 are out of place.

In [19]:
def hamming_distance(board, goal):
    distance = 0
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] != goal[i][j]:
                distance += 1
    return distance

### Manhattan Distance
The Manhattan distances is the sum of the vertical and horizontal distance of the blocks to their goal positions. For example, the Manhattan distance between the blocks in the goal state and the blocks in the state
```
7 2 4
6 0 5
1 3 8
```
is $16$ because <br>
Digits &emsp; &emsp; &emsp; &emsp; &emsp; &nbsp; 1 2 3 4 5 6 7 8 <br> 
Row distance &emsp; &emsp; &nbsp; &nbsp; 2 0 2 1 0 0 2 0 <br>
Column distance &emsp; &nbsp; 2 0 1 2 1 2 0 1 <br>
Total distance &emsp; &emsp; &nbsp; 4 0 3 3 1 2 2 1  = 16 <br>

In [20]:
def manhattan_distance(board, goal):
    distance = 0
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] != goal[i][j] and board[i][j] != 0:
                x, y = divmod(board[i][j] - 1, len(board))
                distance += abs(x - i) + abs(y - j)
    return distance

### Linear Conflict
Two tiles are in linear conflict if both tiles are in their target rows or columns, and in the same setting of the manhattan heuristic, they must pass over each other in order to reach their final goal position. Since we know that in an actual instance of a game, there is no possible way for tiles to actually slide over each other. If such a conflict arises, then one of the tiles would need to move out of the aforementioned row or column, and back in again, adding two moves to the sum of their manhattan distances. Thus the linear conflict heuristic is calculated as, $$ManhattanDistance + 2*(LinearConflicts)$$.
For example, the Linear Conflixt between the blocks in the goal state and the blocks in the state
```
7 2 4
6 0 5
1 3 8
```
is $1$ because $5$ and $6$ are in the same row and they must pass over each other to reach their goal positions. The result of our heuristic will be $ManhattanDistance + 2*LinearConflicts = 16 + 2*1 = 18$.

In [21]:
def linear_conflict(board, goal):
    distance = 0                                                                # initialize distance
    for i in range(len(board)):                                        # iterate through rows                       
        for j in range(len(board[i])):                   # iterate through columns      
            if board[i][j] != goal[i][j] and board[i][j] != 0:      # if the current tile is not in the correct position and is not the blank tile
                x, y = divmod(board[i][j] - 1, len(board))          # get the correct position of the tile
                distance += abs(x - i) + abs(y - j)                 
                if x == i:                                          # if the tile is in the same row as its correct position
                    for k in range(j + 1, len(board[i])):           # iterate through the rest of the row
                        if board[i][k] != goal[i][k] and board[i][k] != 0:  # if the tile is not in the correct position and is not the blank tile
                            x2, y2 = divmod(board[i][k] - 1, len(board))        # get the correct position of the tile
                            if x2 == i and y2 < y:                              # if the tile is in the same row as its correct position and is to the left of the current tile
                                distance += 2                                   # add 2 to the distance
                if y == j:                                          # if the tile is in the same column as its correct position
                    for k in range(i + 1, len(board)):                  # iterate through the rest of the column
                        if board[k][j] != goal[k][j] and board[k][j] != 0:    # if the tile is not in the correct position and is not the blank tile
                            x2, y2 = divmod(board[k][j] - 1, len(board))        # get the correct position of the tile
                            if y2 == j and x2 < x:                    # if the tile is in the same column as its correct position and is above the current tile
                                distance += 2                   # add 2 to the distance
    
    return manhattan_distance(board, goal) + 2*distance

# A* Search
The algorithm works as follows:
1. Initialize the open list
2. Initialize the closed list
3. Add the start node in the open list
4. Loop until you find the end <br>
    a) Pop the node with the least f value from open list, call it "q" <br>
    b) Generate q's successors and set their parents to q <br>
    c) For each successor <br>
        &emsp; i) If successor is the goal, stop the search <br>
        &emsp; ii) If successor is not in the open list <br>
            &emsp; &emsp; i) Compute its f, g and h values <br>
            &emsp; &emsp;ii) Add it to the open list <br>
        &emsp; iii) If successor is already in the open list, check if current path is better than the one already present <br>
            &emsp; &emsp; i) If yes, update the f, g and h values of the node <br>
            &emsp; &emsp; ii) If no, continue with the next successor <br>
    d) Push q on the closed list <br>

In [22]:
def a_star_search(initial_board, goal_board, heuristic):

    expanded_nodes = 0
    explored_nodes = 0
    
    open_list = []
    closed_list = set()

    start_node = PuzzleNode(initial_board)
    start_node.h = heuristic(initial_board, goal_board)
    start_node.f = start_node.g + start_node.h
    heapq.heappush(open_list, start_node)

    while open_list:
        current_node = heapq.heappop(open_list)

        if current_node.board == goal_board:
            return (reconstruct_path(current_node), expanded_nodes, explored_nodes)

        closed_list.add(tuple(map(tuple, current_node.board)))

        neighbors = generate_neighbors(current_node)
        explored_nodes += len(neighbors)

        for neighbor in neighbors:
            if tuple(map(tuple, neighbor.board)) not in closed_list:
                neighbor.g = current_node.g + 1
                neighbor.h = heuristic(neighbor.board, goal_board)
                neighbor.f = neighbor.g + neighbor.h
                heapq.heappush(open_list, neighbor)

        expanded_nodes += 1
    
    return None

# Create Board and Printing Functions

In [23]:
def create_puzzle_board(n):
    numbers = list(range(n * n))
    puzzle_board = [numbers[i:i + n] for i in range(0, len(numbers), n)]
    return puzzle_board

def get_user_input():
    while True:
        try:
            n = int(input("Enter the value of 'n' for N-Puzzle (e.g., 3 for 8-Puzzle): "))
            if n < 2:
                print("Please enter a valid value greater than 1.")
            else:
                initial_board = []
                print("Enter the values for the initial board (0 for blank space): ")
                for i in range(n):
                    row = []
                    for j in range(n):
                        row.append(int(input("Enter the value for row {} and column {}: ".format(i + 1, j + 1))))
                    initial_board.append(row)

                # goal_board = []
                # print("Enter the values for the goal board (0 for blank space): ")
                # for i in range(n):
                #     row = []
                #     for j in range(n):
                #         row.append(int(input("Enter the value for row {} and column {}: ".format(i + 1, j + 1))))
                #     goal_board.append(row)

                goal_board = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

                return n, initial_board, goal_board
        except ValueError:
            print("Invalid input. Please enter a valid integer value.")

def print_puzzle_board(board):
    for row in board:
        for num in row:
            if num == 0:
                print("*", end=" ")
            else:
                print(num, end=" ")
        print()

# Solving the Puzzle

In [28]:
# n, initial_board, goal_board = get_user_input()
n = 3
initial_board = [[2, 7, 5], [1, 8, 4], [0, 6, 3]]
goal_board = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

print("Initial Puzzle:")
print_puzzle_board(initial_board)

print("Goal Puzzle:")
print_puzzle_board(goal_board)

if not is_solvable(n, initial_board):
    print("The puzzle is not solvable.")

Initial Puzzle:
2 7 5 
1 8 4 
* 6 3 
Goal Puzzle:
1 2 3 
4 5 6 
7 8 * 


In [29]:
path, expanded_nodes, explored_nodes = a_star_search(initial_board, goal_board, heuristic=hamming_distance)
if path:
    print("Solution for Hamming Distance:")
    for board in path:
        print_puzzle_board(board)
        print()
    print("Number of expanded nodes: {}".format(expanded_nodes))
    print("Number of explored nodes: {}".format(explored_nodes))

# manhattan distance
path, expanded_nodes, explored_nodes = a_star_search(initial_board, goal_board, heuristic=manhattan_distance)
if path:
    print("Solution for Manhattan Distance:")
    for board in path:
        print_puzzle_board(board)
        print()
    print("Number of expanded nodes: {}".format(expanded_nodes))
    print("Number of explored nodes: {}".format(explored_nodes))

# linear conflict
path, expanded_nodes, explored_nodes = a_star_search(initial_board, goal_board, heuristic=linear_conflict)
if path:
    print("Solution for Linear Conflict:")
    for board in path:
        print_puzzle_board(board)
        print()
    print("Number of expanded nodes: {}".format(expanded_nodes))
    print("Number of explored nodes: {}".format(explored_nodes))

Solution for Hamming Distance:
2 7 5 
1 8 4 
* 6 3 

2 7 5 
* 8 4 
1 6 3 

2 7 5 
8 * 4 
1 6 3 

2 * 5 
8 7 4 
1 6 3 

2 5 * 
8 7 4 
1 6 3 

2 5 4 
8 7 * 
1 6 3 

2 5 4 
8 7 3 
1 6 * 

2 5 4 
8 7 3 
1 * 6 

2 5 4 
8 * 3 
1 7 6 

2 * 4 
8 5 3 
1 7 6 

2 4 * 
8 5 3 
1 7 6 

2 4 3 
8 5 * 
1 7 6 

2 4 3 
8 * 5 
1 7 6 

2 4 3 
* 8 5 
1 7 6 

2 4 3 
1 8 5 
* 7 6 

2 4 3 
1 8 5 
7 * 6 

2 4 3 
1 * 5 
7 8 6 

2 * 3 
1 4 5 
7 8 6 

* 2 3 
1 4 5 
7 8 6 

1 2 3 
* 4 5 
7 8 6 

1 2 3 
4 * 5 
7 8 6 

1 2 3 
4 5 * 
7 8 6 

1 2 3 
4 5 6 
7 8 * 

Number of expanded nodes: 6124
Number of explored nodes: 16484
Solution for Manhattan Distance:
2 7 5 
1 8 4 
* 6 3 

2 7 5 
* 8 4 
1 6 3 

2 7 5 
8 * 4 
1 6 3 

2 7 5 
8 4 * 
1 6 3 

2 7 5 
8 4 3 
1 6 * 

2 7 5 
8 4 3 
1 * 6 

2 7 5 
8 * 3 
1 4 6 

2 7 5 
* 8 3 
1 4 6 

2 7 5 
1 8 3 
* 4 6 

2 7 5 
1 8 3 
4 * 6 

2 7 5 
1 * 3 
4 8 6 

2 * 5 
1 7 3 
4 8 6 

2 5 * 
1 7 3 
4 8 6 

2 5 3 
1 7 * 
4 8 6 

2 5 3 
1 7 6 
4 8 * 

2 5 3 
1 7 6 
4 * 8 

2 5 3 
1 * 6 
4